In [1]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

import gspread
from gspread_dataframe import set_with_dataframe

import pandas as pd

In [2]:
SCOPES = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/spreadsheets"]

def validar_token_google_sheets():
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "client_secret.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())
  return creds


In [3]:
def criar_planilha_gs(nome_planilha):
    creds = validar_token_google_sheets()
    client = gspread.authorize(creds)  # Substitua 'credentials.json' pelo caminho do seu arquivo de credenciais

    # Cria uma nova planilha com o nome fornecido
    nova_planilha = client.create(nome_planilha)

    # Retorna o objeto da planilha recém-criada
    return nova_planilha

In [4]:
def ler_planilha(nome_planilha, nome_pagina):
    creds = validar_token_google_sheets()
    client = gspread.authorize(creds) 

    # Abre a planilha pelo nome
    planilha = client.open(nome_planilha)

    # Seleciona a página desejada
    pagina = planilha.worksheet(nome_pagina)

    # Lê os dados da página como uma lista de listas
    dados = pagina.get_all_values()

    # Retorna os dados
    return dados

In [5]:
def editar_planilha(nome_planilha, nome_pagina, dados):
    # Autentica usando credenciais do Google Cloud Platform
    creds = validar_token_google_sheets()
    client = gspread.authorize(creds)
    # Abre a planilha pelo nome
    planilha = client.open(nome_planilha)

    # Seleciona a página desejada
    pagina = planilha.worksheet(nome_pagina)

    # Limpa os dados existentes na página
    pagina.clear()

    # Preenche a página com os novos dados
    for i, linha in enumerate(dados, start=1):
        pagina.update(f"A{i}", [str(valor) for valor in linha])


In [6]:
def transforma_dataframe_em_lista(dataframe):
    # Transforma o DataFrame em uma lista de listas
    lista = dataframe.values.tolist()

    # Retorna a lista
    return lista

In [1]:
def baixar_pagina_planilha(nome_planilha, nome_pagina):
    # Autentica usando credenciais do Google Cloud Platform
    creds = validar_token_google_sheets()
    client = gspread.authorize(creds) 

    # Abre a planilha pelo nome
    planilha = client.open(nome_planilha)

    # Seleciona a página desejada
    pagina = planilha.worksheet(nome_pagina)

    # Obtém os dados da página como DataFrame
    dataframe = pd.DataFrame(pagina.get_all_records())

    # Retorna o DataFrame
    return dataframe


In [8]:
def remover_copias(df_remove, df_continua):
    print("Len:",len(df_remove),"\nLen:", len(df_continua))
    # Identificar linhas em comum com base no número de inscrição
    inscricoes_comuns = set(df_remove['Inscrição']).intersection(set(df_continua['Inscrição']))

    # Remover as linhas correspondentes do dataframe da página "Tijuca"
    df_remove = df_remove[~df_remove['Inscrição'].isin(inscricoes_comuns)]

    print("Len:",len(df_remove),"\nLen:", len(df_continua))
    return df_remove



In [9]:
def remover_planilha_adicionar_df(nome_planilha, nome_pagina, novo_df):
    creds = validar_token_google_sheets()
    client = gspread.authorize(creds)
    spreadsheet = client.open(nome_planilha)
    worksheet = spreadsheet.worksheet(nome_pagina)

    # Limpar os dados na planilha
    worksheet.clear()
    # Atualizar a planilha com os dados atualizados
    novo_df_list = transforma_dataframe_em_lista(novo_df, add_columns=True)
    worksheet.append_rows(novo_df_list)
    print("Planilha atualizada")